# The data
The file contains 1,00,000 records.In whichit contains train data and test data. The review column contains text for the review . The train and test data both contain  "positive" and "negative" review. Through this data we provide positive and negative sentiment which makes our problem a binary classification problem.


# Importing Required Libraries
The following script imports the required libraries:

In [84]:
import pandas as pd
import numpy as np
import re
import os
from IPython.display import HTML

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.decomposition import PCA

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import words
from nltk.corpus import wordnet 
allEnglishWords = words.words() + [w for w in wordnet.words()]
allEnglishWords = np.unique([x.lower() for x in allEnglishWords])

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

# Importing and Analyzing the Dataset
Let's now import and analyze our dataset. Execute the following script

In [3]:

path='E:\project_minor'
positiveFiles=[x for x in os.listdir(path + '/train_pos/') if x.endswith(".txt")]
negativeFiles=[x for x in os.listdir(path + '/train_neg/') if x.endswith(".txt")]
TpositiveFiles=[x for x in os.listdir(path + '/test_pos/') if x.endswith(".txt")]
TnegativeFiles=[x for x in os.listdir(path + '/test_neg/') if x.endswith(".txt")]



# Append train data and test data & label that review 
Here 50k train (.txt)data & 50k test (.txt) file and we append all of them . providing label 0 as positive ,label 1 for train data.label -1 for test data.

In [7]:
positiveReviews, negativeReviews, tpestReviews,tnestReviews = [], [], [],[]
for pfile in positiveFiles:
    with open(path+'/train_pos/'+pfile, encoding="latin1") as f:
        positiveReviews.append(f.read())
for nfile in negativeFiles:
    with open(path+'/train_neg/'+nfile, encoding="latin1") as f:
        negativeReviews.append(f.read())
for tfile in TpositiveFiles:
    with open(path+'/test_pos/'+tfile, encoding="latin1") as f:
        tpestReviews.append(f.read())
for tfile in TnegativeFiles:
    with open(path+'/test_neg/'+tfile, encoding="latin1") as f:
        tnestReviews.append(f.read())

# print
reviews.head() prints the data.first 5 rows of the dataset using the head() method.


In [8]:
reviews = pd.concat([
    pd.DataFrame({"review":positiveReviews, "label":1, "file":positiveFiles}),
    pd.DataFrame({"review":negativeReviews, "label":0, "file":negativeFiles}),
    pd.DataFrame({"review":positiveReviews, "label":-1, "file":TpositiveFiles}),
    pd.DataFrame({"review":negativeReviews, "label":-1, "file":TnegativeFiles})
], ignore_index=True).sample(frac=1, random_state=1)
reviews.head()

,review,label,file
26247,"When we started watching this series on cable,...",-1,11122_8.txt
35067,Steve Biko was a black activist who tried to r...,-1,7811_10.txt
34590,My short comment for this flick is go pick it ...,-1,7382_10.txt
16668,I saw this on TV the other nightÂ or rather I...,0,2501_1.txt
12196,I am a huge fan of Simon Pegg and have watched...,1,9728_7.txt


In [11]:
reviews = reviews[["review", "label", "file"]].sample(frac=1, random_state=1)
train = reviews[reviews.label!=-1].sample(frac=0.6, random_state=1)
valid = reviews[reviews.label!=-1].drop(train.index)
test = reviews[reviews.label==-1]

# print shape of data

In [12]:
print(train.shape)
print(valid.shape)
print(test.shape)

(15000, 3)
(10000, 3)
(25000, 3)


# HTML tags will preprocess this text in the next section.

In [13]:
HTML(train.review.iloc[0])

# Data Preprocessing
We saw that our dataset contained punctuations and HTML tags. In this section we will define a function that takes a text string as a parameter and then performs preprocessing on the string to remove special characters and HTML tags from the string. Finally, the string is returned to the calling function



In [14]:
class Preprocessor(object):
    ''' Preprocess data for NLP tasks. '''

    def __init__(self, alpha=True, lower=True, stemmer=True, english=False):
        self.alpha = alpha
        self.lower = lower
        self.stemmer = stemmer
        self.english = english
        
        self.uniqueWords = None
        self.uniqueStems = None
        
    def fit(self, texts):
        texts = self._doAlways(texts)

        allwords = pd.DataFrame({"word": np.concatenate(texts.apply(lambda x: x.split()).values)})
        self.uniqueWords = allwords.groupby(["word"]).size().rename("count").reset_index()
        self.uniqueWords = self.uniqueWords[self.uniqueWords["count"]>1]
        if self.stemmer:
            self.uniqueWords["stem"] = self.uniqueWords.word.apply(lambda x: PorterStemmer().stem(x)).values
            self.uniqueWords.sort_values(["stem", "count"], inplace=True, ascending=False)
            self.uniqueStems = self.uniqueWords.groupby("stem").first()
        
        #if self.english: self.words["english"] = np.in1d(self.words["mode"], allEnglishWords)
        print("Fitted.")
            
    def transform(self, texts):
        texts = self._doAlways(texts)
        if self.stemmer:
            allwords = np.concatenate(texts.apply(lambda x: x.split()).values)
            uniqueWords = pd.DataFrame(index=np.unique(allwords))
            uniqueWords["stem"] = pd.Series(uniqueWords.index).apply(lambda x: PorterStemmer().stem(x)).values
            uniqueWords["mode"] = uniqueWords.stem.apply(lambda x: self.uniqueStems.loc[x, "word"] if x in self.uniqueStems.index else "")
            texts = texts.apply(lambda x: " ".join([uniqueWords.loc[y, "mode"] for y in x.split()]))
        #if self.english: texts = self.words.apply(lambda x: " ".join([y for y in x.split() if self.words.loc[y,"english"]]))
        print("Transformed.")
        return(texts)

    def fit_transform(self, texts):
        texts = self._doAlways(texts)
        self.fit(texts)
        texts = self.transform(texts)
        return(texts)
    
    def _doAlways(self, texts):
        # Remove parts between <>'s
        texts = texts.apply(lambda x: re.sub('<.*?>', ' ', x))
        # Keep letters and digits only.
        if self.alpha: texts = texts.apply(lambda x: re.sub('[^a-zA-Z0-9 ]+', ' ', x))
        # Set everything to lower case
        if self.lower: texts = texts.apply(lambda x: x.lower())
        return texts  

In [15]:
train.head()

,review,label,file
15710,The earlier part of the film was rather enjoya...,0,163_4.txt
14400,There is only one reason to watch this movie i...,0,11710_4.txt
313,"If your idea of a thriller is car chases, expl...",1,10282_8.txt
14601,"Sadly, the print of the film we were going to ...",0,11892_4.txt
20691,Surely one of the most ill-advised remakes of ...,0,6122_2.txt


In [17]:
preprocess = Preprocessor(alpha=True, lower=True, stemmer=True)

In [18]:
%%time
trainX = preprocess.fit_transform(train.review)
validX = preprocess.transform(valid.review)

Fitted.
Transformed.
Transformed.
Wall time: 57 s


In [19]:
trainX.head()

15710    the earlier part of the film was rather enjoy ...
14400    there is only one reason to watch this movie i...
313      if your idea of a thriller is car chase explos...
14601    sadly the print of the film we were go to watc...
20691    sure one of the most ill advise remake of a cl...
Name: review, dtype: object

In [20]:
print(preprocess.uniqueWords.shape)
preprocess.uniqueWords[preprocess.uniqueWords.word.str.contains("disappoint")]

(38409, 3)


,word,count,stem
15085,disappointingly,8,disappointingli
15083,disappointed,581,disappoint
15084,disappointing,257,disappoint
15086,disappointment,245,disappoint
15082,disappoint,62,disappoint
15089,disappoints,23,disappoint
15088,disappointments,14,disappoint


In [21]:
print(preprocess.uniqueStems.shape)
preprocess.uniqueStems[preprocess.uniqueStems.word.str.contains("disappoint")]

(25433, 2)


,word,count
stem,,
disappoint,disappointed,581
disappointingli,disappointingly,8


# Data Preprocessing
We saw that our dataset contained punctuations and HTML tags. In this section we will define a function that takes a text string as a parameter and then performs preprocessing on the string to remove special characters and HTML tags from the string. Finally, the string is returned to the calling function



In [22]:
stop_words = text.ENGLISH_STOP_WORDS.union(["thats","weve","dont","lets","youre","im","thi","ha",
    "wa","st","ask","want","like","thank","know","susan","ryan","say","got","ought","ive","theyre"])
tfidf = TfidfVectorizer(min_df=2, max_features=10000, stop_words=stop_words) #, ngram_range=(1,3)

In [23]:
%%time
trainX = tfidf.fit_transform(trainX).toarray()
validX = tfidf.transform(validX).toarray()

Wall time: 4.45 s


In [24]:
print(trainX.shape)
print(validX.shape)

(15000, 10000)
(10000, 10000)


In [39]:
trainY = np.array(train.label)
validY = np.array(valid.label)

In [40]:
print(trainX.shape, trainY.shape)
print(validX.shape, validY.shape)

(15000, 2000) (15000,)
(10000, 2000) (10000,)


# Feature Selection
Next, we take the 10k dimensional tfidf's as input, and keep the 2000 dimensions that correlate the most with our sentiment target. The corresponding words - see below - make sense.

In [41]:
from scipy.stats.stats import pearsonr

In [42]:
getCorrelation = np.vectorize(lambda x: pearsonr(trainX[:,x], trainY)[0])
correlations = getCorrelation(np.arange(trainX.shape[1]))
print(correlations)

[ 0.19451991  0.16715002  0.15920693 ... -0.20991894 -0.23274205
 -0.26436818]


In [82]:
allIndeces = np.argsort(-correlations)
bestIndeces = allIndeces[np.concatenate([np.arange(1000), np.arange(-1000, 0)])]

In [83]:
vocabulary = np.array(tfidf.get_feature_names())
print(vocabulary[bestIndeces][:10])
print(vocabulary[bestIndeces][-10:])


['00' '000' '01' '02' '10' '100' '1000' '101' '11' '11th']
['content' 'contest' 'context' 'continent' 'continue' 'contract'
 'contradiction' 'contrary' 'contrast' 'contribute']


In [45]:
trainX = trainX[:,bestIndeces]
validX = validX[:,bestIndeces]


In [46]:
print(trainX.shape, trainY.shape)
print(validX.shape, validY.shape)

(15000, 2000) (15000,)
(10000, 2000) (10000,)


# Model Architecture:Text Classification with Simple Deep Neural Network

We choose a very simple dense network with 6 layers, performing binary classification.

we create a Sequential() model. Next, we create our embedding layer. The embedding layer will have an input length & the output vector dimension. The vocabulary size will be 92547 words. Since we are not training our own embeddings and using the GloVe embedding, we set trainable to False and in the weights attribute we pass our own embedding matrix.

The embedding layer is then added to our model. Next, since we are directly connecting our embedding layer to densely connected layer, we flatten the embedding layer. Finally, we add a dense layer with sigmoid activation function.

In [47]:
DROPOUT = 0.5
ACTIVATION = "tanh"

model = Sequential([    
    Dense(int(trainX.shape[1]/2), activation=ACTIVATION, input_dim=trainX.shape[1]),
    Dropout(DROPOUT),
    Dense(int(trainX.shape[1]/2), activation=ACTIVATION, input_dim=trainX.shape[1]),
    Dropout(DROPOUT),
    Dense(int(trainX.shape[1]/4), activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(100, activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(20, activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(5, activation=ACTIVATION),
    Dropout(DROPOUT),
    Dense(1, activation='sigmoid'),
])


# To compile
our model, we will use the adam optimizer, binary_crossentropy as our loss function and accuracy as metrics and then we will print the summary of our model:

In [1]:
model.compile(optimizer=optimizers.Adam(0.00005), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


NameError: name 'model' is not defined

# Model Training
Let's go.

we use the fit method to train our neural network. Notice we are training on our train set only. The validation_split of 0.2 means that 20% of the training data is used to find the training accuracy of the algorithm.

At the end of the training, you will see that training accuracy is around 88.25%.

Training occurs over epochs and each epoch is split into batches.

Epoch: One pass through all of the rows in the training dataset.

Batch: One or more samples considered by the model within an epoch before weights are updated.


In [49]:
EPOCHS = 30
BATCHSIZE = 1500


In [50]:

model.fit(trainX, trainY, epochs=EPOCHS, batch_size=BATCHSIZE, validation_data=(validX, validY))

Train on 15000 samples, validate on 10000 samples
Epoch 1/30
15000/15000 [==============================] - 4s 299us/sample - loss: 0.7020 - accuracy: 0.5111 - val_loss: 0.6817 - val_accuracy: 0.6694
Epoch 2/30
15000/15000 [==============================] - 2s 160us/sample - loss: 0.6949 - accuracy: 0.5191 - val_loss: 0.6695 - val_accuracy: 0.7414
Epoch 3/30
15000/15000 [==============================] - 2s 160us/sample - loss: 0.6784 - accuracy: 0.5634 - val_loss: 0.6560 - val_accuracy: 0.7703
Epoch 4/30
15000/15000 [==============================] - 2s 159us/sample - loss: 0.6652 - accuracy: 0.5889 - val_loss: 0.6395 - val_accuracy: 0.7898
Epoch 5/30
15000/15000 [==============================] - 3s 184us/sample - loss: 0.6515 - accuracy: 0.6095 - val_loss: 0.6191 - val_accuracy: 0.8005
Epoch 6/30
15000/15000 [==============================] - 2s 162us/sample - loss: 0.6304 - accuracy: 0.6459 - val_loss: 0.5938 - val_accuracy: 0.8100
Epoch 7/30
15000/15000 [==========================

# To check the test accuracy and loss
To evaluate the performance of the model, we can simply pass the test set to the evaluate method of our model.

In [51]:
train["probability"] = model.predict(trainX)
train["prediction"] = train.probability-0.5>0
train["truth"] = train.label==1
train.tail()

,review,label,file,probability,prediction,truth
8573,Two movies back to back which dealt with India...,1,6467_7.txt,0.728238,True,True
2696,This film has a lot of raw potential. The scri...,1,12427_7.txt,0.861215,True,True
11256,"John Singleton's finest film, before blockbust...",1,8882_10.txt,0.890667,True,True
2906,am i the only one who saw the connection betwe...,1,1366_8.txt,0.250345,False,True
17972,"Susan Swift is an appealing youngster, a flowe...",0,3676_2.txt,0.367998,False,False


# plot
plot the loss and accuracy differences for training and test sets.

In [52]:
x = np.arange(EPOCHS)
history = model.history.history

data = [
    go.Scatter(x=x, y=history["accuracy"], name="Train Accuracy", marker=dict(size=5), yaxis='y2'),
    go.Scatter(x=x, y=history["val_accuracy"], name="Valid Accuracy", marker=dict(size=5), yaxis='y2'),
    go.Scatter(x=x, y=history["loss"], name="Train Loss", marker=dict(size=5)),
    go.Scatter(x=x, y=history["val_loss"], name="Valid Loss", marker=dict(size=5))
]
layout = go.Layout(
    title="Model Training Evolution", font=dict(family='Palatino'), xaxis=dict(title='Epoch', dtick=1),
    yaxis1=dict(title="Loss", domain=[0, 0.45]), yaxis2=dict(title="Accuracy", domain=[0.55, 1]),
)
py.iplot(go.Figure(data=data, layout=layout), show_link=False)

# To evaluate the performance of the model
we can simply pass the test set to the evaluate method of our model.

In [53]:
print(model.evaluate(trainX, trainY))
print((train.truth==train.prediction).mean())

15000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.2546879736105601, 0.9242667]
0.9242666666666667


In [54]:
valid["probability"] = model.predict(validX)
valid["prediction"] = valid.probability-0.5>0
valid["truth"] = valid.label==1
valid.tail()

,review,label,file,probability,prediction,truth
20070,Probably the worst Dolph film ever. There's no...,0,5564_2.txt,0.102339,False,False
8612,If you want just about everything you want to ...,1,6501_10.txt,0.883922,True,True
3341,I was really surprised with this movie. Going ...,1,1758_10.txt,0.105403,False,True
22644,This film is a portrait of the half-spastic te...,0,7881_2.txt,0.111720,False,False
10802,This charmingly pleasant and tenderhearted seq...,1,8473_8.txt,0.855971,True,True


In [55]:
print(model.evaluate(validX, validY))
print((valid.truth==valid.prediction).mean())

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.33167930936813356, 0.8725]
0.8725


# Error Analysis
Error analysis gives us great insight in the way the model is making its errors. Often, it shows data quality issues.

In [56]:
trainCross = train.groupby(["prediction", "truth"]).size().unstack()
trainCross

truth,False,True
prediction,,
False,6878,550
True,586,6986


In [57]:
validCross = valid.groupby(["prediction", "truth"]).size().unstack()
validCross

truth,False,True
prediction,,
False,4360,599
True,676,4365


In [58]:
truepositives = valid[(valid.truth==True)&(valid.truth==valid.prediction)]
print(len(truepositives), "true positives.")
truepositives.sort_values("probability", ascending=False).head(3)

4365 true positives.


,review,label,file,probability,prediction,truth
7809,This is one of the most beautiful films I have...,1,577_8.txt,0.899320,True,True
10694,One of the best movies out there. Yeah maybe t...,1,8376_10.txt,0.899320,True,True
9005,This is a taut suspenseful masterpiece from Br...,1,6856_10.txt,0.899281,True,True


In [59]:
truenegatives = valid[(valid.truth==False)&(valid.truth==valid.prediction)]
print(len(truenegatives), "true negatives.")
truenegatives.sort_values("probability", ascending=True).head(3)

4360 true negatives.


,review,label,file,probability,prediction,truth
24238,This movie was so bad! It was terrible! It was...,0,9315_1.txt,0.100664,False,False
18791,"I would have given it a one instead of a two, ...",0,4412_2.txt,0.100669,False,False
19045,I sometimes enjoy really lousy movies....those...,0,4641_1.txt,0.100702,False,False


In [60]:
falsepositives = valid[(valid.truth==True)&(valid.truth!=valid.prediction)]
print(len(falsepositives), "false positives.")
falsepositives.sort_values("probability", ascending=True).head(3)

599 false positives.


,review,label,file,probability,prediction,truth
12340,I wouldn't go so far as to not recommend this ...,1,9858_7.txt,0.101624,False,True
7172,A bunch of American students and their tutor d...,1,5205_7.txt,0.101944,False,True
972,I was really excited about seeing this film. I...,1,10876_7.txt,0.102543,False,True


In [61]:
falsenegatives = valid[(valid.truth==False)&(valid.truth!=valid.prediction)]
print(len(falsenegatives), "false negatives.")
falsenegatives.sort_values("probability", ascending=False).head(3)

676 false negatives.


,review,label,file,probability,prediction,truth
14449,this is a great movie. I love the series on tv...,0,11755_3.txt,0.898370,True,False
14715,I was so excited and hyped up about watching t...,0,11995_4.txt,0.897589,True,False
19440,this one of the best celebrity's reality shows...,0,4998_3.txt,0.896916,True,False


In [62]:
HTML(valid.loc[14449].review)

# Model Application
Custom Reviews To use this model, we would store the model, along with the preprocessing vectorizers, and run the unseen texts through following pipeline.

In [104]:
unseen = pd.Series("this  is a good movie")

In [105]:
unseen = preprocess.transform(unseen)       # Text preprocessing
unseen = tfidf.transform(unseen).toarray()  # Feature engineering
unseen = unseen[:,bestIndeces]              # Feature selection
probability = model.predict(unseen)[0,0]  # Network feedforward

Transformed.


In [106]:
print(probability)
print("Positive!") if probability > 0.5 else print("Negative!")

0.5739988
Positive!


In [122]:
unseen = pd.Series("this  is a bad movie")

In [123]:
unseen = preprocess.transform(unseen)       # Text preprocessing
unseen = tfidf.transform(unseen).toarray()  # Feature engineering
unseen = unseen[:,bestIndeces]              # Feature selection
probability = model.predict(unseen)[0,0]  # Network feedforward

Transformed.


In [124]:
print(probability)
print("Positive!") if probability > 0.5 else print("Negative!")

0.33640224
Negative!


In [116]:
unseen = pd.Series(" a great movie")

In [117]:
unseen = preprocess.transform(unseen)       # Text preprocessing
unseen = tfidf.transform(unseen).toarray()  # Feature engineering
unseen = unseen[:,bestIndeces]              # Feature selection
probability = model.predict(unseen)[0,0]  # Network feedforward

Transformed.


In [118]:
print(probability)
print("Positive!") if probability > 0.5 else print("Negative!")

0.5739988
Positive!
